In [ ]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
basedir="/Users/tgadf/Downloads/stateshapes"

In [ ]:
from os.path import basename, dirname
shapeval = "Amtrak_Stations"
try:
    sf = shapefile.Reader(join(basedir, shapeval, shapeval))
except:
    raise ValueError("No shapefile!")
fields      = sf.fields
shapeData   = {}
geoShapeMap = {}
Nshapes   = len(sf.shapes())
ngeos     = 0
totalgeos = 0
prec      = 7
show      = False

start,cmt = clock("Analyzing {0}\t{1}".format(shapeval, Nshapes))
if show:
    print("\n\nFields -> {0}".format(fields))


irec = -1
for shapeRec in sf.iterShapeRecords():
    irec += 1

    ## Record
    record = shapeRec.record
    if show: raise ValueError("Stopping here: {0}".format(record))
    geoid = record[1]
    name  = record[2]
    shapeData[geoid] = {"Name": name, "Record": irec}
        
    geos  = getGeos(shapeRec.shape, irec, Nshapes, prec=7)

    
    if len(geos) > 0:
        newgeos = set()
        rad = 100
        long, lat = shapeRec.shape.points[0]
        nmiss = 0
        for i in range(10000):
            dLg = uniform(-rad, rad) / 111000
            dLa = uniform(-rad, rad) / 111000
            pnt = (lat + dLa, long + dLg)
            dist = haversine((lat, long), pnt)
            if dist < rad/1000:
                geo    = geohash.encode(latitude=pnt[0], longitude=pnt[1], precision=7)
                if geo not in newgeos:
                    newgeos.add(geo)
                    nmiss = 0
                else:
                    nmiss += 1

            if nmiss > 200:
                break

        geoShapeMap[geoid] = newgeos
        ngeos += len(newgeos)

print("Found {0} geos from {1}".format(ngeos, shapeval))
saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, "{0}-{1}".format(shapeval, 7))

In [ ]:
from os.path import basename, dirname
shapeval = "Airports"
try:
    sf = shapefile.Reader(join(basedir, shapeval, shapeval))
except:
    raise ValueError("No shapefile!")
fields      = sf.fields
shapeData   = {}
geoShapeMap = {}
Nshapes   = len(sf.shapes())
ngeos     = 0
totalgeos = 0
prec      = 8
show      = False

start,cmt = clock("Analyzing {0}\t{1}".format(shapeval, Nshapes))
if show:
    print("\n\nFields -> {0}".format(fields))


iname=None
for i,val in enumerate(fields):
    if val[0] == "FullName":
        iname = i-1
        break
itype=None
for i,val in enumerate(fields):
    if val[0] == "OwnerType":
        itype = i-1
        break
iacres=None
for i,val in enumerate(fields):
    if val[0] == "Acres":
        iacres = i-1
        break

irec = -1
from numpy import sqrt
from haversine import haversine
from random import uniform
for shapeRec in sf.iterShapeRecords():
    irec += 1
    if irec % 1000 == 0:
        print(irec)

    ## Record
    record = shapeRec.record
    if show: raise ValueError("Stopping here: {0}".format(record))
    geoid = record[1]
    name  = record[iname]
    size  = record[iacres]
    atype = record[itype]
    shapeData[geoid] = {"Name": name, "Record": irec, "Type": atype, "Size": size}
        
    geos  = getGeos(shapeRec.shape, irec, Nshapes, prec=7)

    if size is not None:
        newgeos = set()
        rad = 63.6167*sqrt(size)/2
        long, lat = shapeRec.shape.points[0]
        nmiss = 0
        for i in range(10000):
            dLg = uniform(-rad, rad) / 111000
            dLa = uniform(-rad, rad) / 111000
            pnt = (lat + dLa, long + dLg)
            dist = haversine((lat, long), pnt)
            if dist < rad/1000:
                geo    = geohash.encode(latitude=pnt[0], longitude=pnt[1], precision=7)
                if geo not in newgeos:
                    newgeos.add(geo)
                    nmiss = 0
                else:
                    nmiss += 1

            if nmiss > 200:
                break
            
        geoShapeMap[geoid] = newgeos
        ngeos += len(newgeos)
    else:
        geoShapeMap[geoid] = geos
        ngeos += len(geos)

print("Found {0} geos from {1}".format(ngeos, shapeval))
saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, "{0}-{1}".format(shapeval, 7))